Throughout this lesson, you've been trying different models on the same two
datasets, wine and diabetes. Now, we're going to try our hand at accelerating
this methodology by using AutoGluon. In this exercise, train two different
AutonGluon models and see how they compare to previous iterations in exercise 1
and 2.

You're tasked with completing the following steps:

1. Load in the wine dataset from Scikit Learn.
2. For the wine dataset, create a train and test split, 80% train / 20% test.
3. Create an AutoGluon Classifier model with these hyper parameters:
    1. `time_limit`: `120`
    2. `presets`: `best_quality`
4. Output the model table summary.
5. Evaluate the trained model on the test dataset.
6. Load the diabetes dataset from Scikit Learn.
7. For the diabetes dataset, create a train and test split, 80% train / 20% test.
8. Create an AutoGluon Regression model with these hyper parameters:
    1. `eval_metric`: `r2`
    2. `time_limit`: `120`
    3. `presets`: `best_quality`
9. Output the model table summary.
10. Evaluate the trained model on the test dataset.

## Setup

### Open up Sagemaker Studio

1. Notebook should be using a `ml.t3.medium` instance (2 vCPU + 4 GiB)
2. Notebook should be using kernel: `Python 3 (MXNet 1.8 Python 3.7 CPU Optimized)`

In [ ]:
!pip install -U pip
!pip install -U setuptools wheel
!pip install -U "mxnet<2.0.0" bokeh==2.0.1
!pip install autogluon --no-cache-dir

In [1]:
import numpy as np
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn import datasets
from sklearn.metrics import accuracy_score, r2_score
from sklearn.model_selection import train_test_split

## AutoGluon Classifier

In [2]:
# Load in the wine dataset
wine = datasets.load_wine()

In [3]:
# Create the wine `data` dataset as a dataframe and name the columns with
# `feature_names`
df = pd.DataFrame(wine["data"], columns=wine["feature_names"])

# Include the target as well
df["target"] = wine["target"]

In [4]:
# Split your data with these ratios: train: 0.8 | test: 0.2
df_train, df_test = train_test_split(df, test_size=0.2, random_state=0)

In [5]:
# How does the model perform on the training dataset and default model
# parameters? Using the hyperparameters in the requirements, is there
# improvement? Remember we use the test dataset to score the model. No need to
# explicitly say this is a classifier, AutoGluon will pick it up.
predictor = TabularPredictor(label="target") \
    .fit(train_data=df_train, time_limit=120, presets="best_quality")

No path specified. Models will be saved in: "AutogluonModels/ag-20220110_214459/"
Presets specified: ['best_quality']
Beginning AutoGluon training ... Time limit = 120s
AutoGluon will save models to "AutogluonModels/ag-20220110_214459/"
AutoGluon Version:  0.3.1
Train Data Rows:    142
Train Data Columns: 13
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	3 unique label values:  [2, 1, 0]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Train Data Class Count: 3
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    2620.64 MB
	Train Data (Original)  Memory Usage: 0.01 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set

In [6]:
# Output the fit summary of the training run
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   RandomForestGini_BAG_L1   0.985915       0.117629   0.785026                0.117629           0.785026            1       True          4
1    NeuralNetFastAI_BAG_L1   0.985915       0.539135  27.118259                0.539135          27.118259            1       True          3
2       WeightedEnsemble_L2   0.985915       0.539761  27.383442                0.000626           0.265183            2       True         11
3           CatBoost_BAG_L1   0.978873       0.060538   8.553924                0.060538           8.553924            1       True          6
4   RandomForestEntr_BAG_L1   0.978873       0.108949   0.726085                0.108949           0.726085            1       True          5
5     ExtraTreesEntr_BAG_L1   0.978873       0.112620   0.601538                

{'model_types': {'KNeighborsUnif_BAG_L1': 'StackerEnsembleModel_KNN',
  'KNeighborsDist_BAG_L1': 'StackerEnsembleModel_KNN',
  'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'RandomForestGini_BAG_L1': 'StackerEnsembleModel_RF',
  'RandomForestEntr_BAG_L1': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'ExtraTreesGini_BAG_L1': 'StackerEnsembleModel_XT',
  'ExtraTreesEntr_BAG_L1': 'StackerEnsembleModel_XT',
  'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'NeuralNetMXNet_BAG_L1': 'StackerEnsembleModel_TabularNeuralNet',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel'},
 'model_performance': {'KNeighborsUnif_BAG_L1': 0.6619718309859155,
  'KNeighborsDist_BAG_L1': 0.7112676056338029,
  'NeuralNetFastAI_BAG_L1': 0.9859154929577465,
  'RandomForestGini_BAG_L1': 0.9859154929577465,
  'RandomForestEntr_BAG_L1': 0.9788732394366197,
  'CatBoost_BAG_L1': 0.9788732394366197,
  'ExtraTreesGini_BAG_L1': 0.9788732394366197,
  'ExtraTrees

In [7]:
# Evaluate the models performance on the test dataset
performance = predictor.evaluate(df_test)

Evaluation: accuracy on test data: 1.0
Evaluations on test data:
{
    "accuracy": 1.0,
    "balanced_accuracy": 1.0,
    "mcc": 1.0
}


## AutoGluon Regression

In [8]:
# Load in the diabetes dataset
diabetes = datasets.load_diabetes()

In [9]:
# Create the diabetes `data` dataset as a dataframe and name the columns with
# `feature_names`
dfd = pd.DataFrame(diabetes["data"], columns=diabetes["feature_names"])

# Include the target as well
dfd["target"] = diabetes["target"]

In [10]:
# Split your data with these ratios: train: 0.8 | test: 0.2
dfd_train, dfd_test = train_test_split(dfd, test_size=0.2, random_state=0)

In [11]:
# How does the model perform on the training dataset and default model
# parameters? Using the hyperparameters in the requirements, is there
# improvement? Remember we use the test dataset to score the model. No need to
# explicitly say this is a regression, AutoGluon will pick it up.
predictor = TabularPredictor(label="target", eval_metric="r2") \
    .fit(train_data=dfd_train, time_limit=120, presets="best_quality")

No path specified. Models will be saved in: "AutogluonModels/ag-20220110_214644/"
Presets specified: ['best_quality']
Beginning AutoGluon training ... Time limit = 120s
AutoGluon will save models to "AutogluonModels/ag-20220110_214644/"
AutoGluon Version:  0.3.1
Train Data Rows:    353
Train Data Columns: 10
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (346.0, 25.0, 151.60623, 78.40991)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    2687.28 MB
	Train Data (Original)  Memory Usage: 0.03 MB (0.0% of available memory)
	Inferring data type of each feature based on colu

In [12]:
# Output the fit summary of the training run
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                    model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0     WeightedEnsemble_L2   0.525215       2.637597  74.078106                0.000580           0.301894            2       True          9
1         CatBoost_BAG_L1   0.523150       0.038673   7.081758                0.038673           7.081758            1       True          4
2    ExtraTreesMSE_BAG_L1   0.502246       0.119874   0.605108                0.119874           0.605108            1       True          5
3  NeuralNetFastAI_BAG_L1   0.498088       0.281615  11.775291                0.281615          11.775291            1       True          6
4   NeuralNetMXNet_BAG_L1   0.491805       2.300716  54.902622                2.300716          54.902622            1       True          8
5  RandomForestMSE_BAG_L1   0.466743       0.117972   0.720549                0.117972      

{'model_types': {'KNeighborsUnif_BAG_L1': 'StackerEnsembleModel_KNN',
  'KNeighborsDist_BAG_L1': 'StackerEnsembleModel_KNN',
  'RandomForestMSE_BAG_L1': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'ExtraTreesMSE_BAG_L1': 'StackerEnsembleModel_XT',
  'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'NeuralNetMXNet_BAG_L1': 'StackerEnsembleModel_TabularNeuralNet',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel'},
 'model_performance': {'KNeighborsUnif_BAG_L1': 0.43052099899418794,
  'KNeighborsDist_BAG_L1': 0.43970122394744415,
  'RandomForestMSE_BAG_L1': 0.4667434753086127,
  'CatBoost_BAG_L1': 0.5231497544217285,
  'ExtraTreesMSE_BAG_L1': 0.502246448213272,
  'NeuralNetFastAI_BAG_L1': 0.4980876886527339,
  'XGBoost_BAG_L1': 0.4521637200377172,
  'NeuralNetMXNet_BAG_L1': 0.4918045287008994,
  'WeightedEnsemble_L2': 0.5252154391098123},
 'model_best': 'WeightedEnsemble_L2',
 'mo

In [13]:
# Evaluate the models performance on the test dataset
performance = predictor.evaluate(dfd_test)

Evaluation: r2 on test data: 0.32045118569633957
Evaluations on test data:
{
    "r2": 0.32045118569633957,
    "root_mean_squared_error": -59.03115052813201,
    "mean_squared_error": -3484.67673267498,
    "mean_absolute_error": -44.845230788327335,
    "pearsonr": 0.5775108744156171,
    "median_absolute_error": -36.25103759765625
}
